<a href="https://colab.research.google.com/github/llavkush/HelloWorld/blob/master/Natural_Language_Understanding__Lavkush.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Problem Statement:
XYZ is a call audit company that audits the promotional call of their employees. They perform the call
audits manually where a person listens all the recorded call and decide whether all the promotional
offers are detailed correctly to the customer or not. But this process is time consuming and involved
lots of manual intervention. So the company decide to use Natural Language Understanding and
develops a product where they deal with processing audio calls, convert the speech into text,
identifying intent and entities from a text and generate the summary for the audio calls. 

In [ ]:
!pip install sklearn_crfsuite --quiet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install rasa_nlu spacy --quiet

In [ ]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

In [ ]:
#Reading Data Sets
train_data = load_data('/content/rasa-dataset.json') #Reading Data Set for training intent Classifier

In [ ]:
# language: "en"
# pipeline: "spacy_sklearn"

In [ ]:
trainer = Trainer(config.load("/content/config_spacy.yaml")) #Reading config file

##Content of config_spacy.yaml

language: "en"

pipeline:
- name: "nlp_spacy"
- name: "tokenizer_spacy"
- name: "intent_entity_featurizer_regex"
- name: "intent_featurizer_spacy"
- name: "ner_crf"
- name: "ner_synonyms"
- name: "intent_classifier_sklearn"

###Model training

In [ ]:
trainer.train(train_data)

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_

In [ ]:
#Saving Model
model_directory = trainer.persist('/content/')

In [ ]:
#Entity extraction with SpaCy
import spacy
nlp = spacy.load('en')

In [ ]:
#Loading a Samle text 
text = "My name is and I am calling from."
docx = nlp('My name is Jeff This is a call regarding your Google Cloud Platform account')
docx = nlp(text)
for word in docx.ents:
  print(word)
    #print("value",word.text,"entity",word.label_,"start",word.start_char,"end",word.end_char)

In [ ]:
#Making predictions with the model
from rasa_nlu.model import Metadata, Interpreter

# where `model_directory points to the folder the model is persisted in
interpreter = Interpreter.load(model_directory)

In [ ]:
# Prediction of Intent
dict_ = interpreter.parse('This is a call regarding your Google Cloud Platform account')
dict_['intent']['name']

'Purpose'

In [ ]:
#Helppr Funtion to Identify Intent And Entity

def cleanNullTerms(d):
   clean = {}
   for k, v in d.items():
      if isinstance(v, dict):
         nested = cleanNullTerms(v)
         if len(nested.keys()) > 0:
            clean[k] = nested
      elif v is not None:
         clean[k] = v
   return clean


def entity_recog(text):
  docx = nlp(text)
  dict_ = {}
  for word in docx.ents:
      dict_[word.label_] = word.text
  dict_ = {'name': dict_.get('PERSON', None),'org': dict_.get('ORG', None)}
  dict_ = cleanNullTerms(dict_)
  d = []
  for k, v in dict_.items():
    if len(dict_) > 1:
      if k == 'name':
        d.append({"entity_name" : "caller_name", "entity_value" : v})
      elif k == 'org':
        d.append({"entity_name" : "company", "entity_value" : v})
    else:
      if k == 'name':
        d.append({"entity_name" : "caller_name", "entity_value" : v})
      else:
        if k == 'org':
          d.append({"entity_name" : "product", "entity_value" : v})
  
  return d

def intent_recog(text):
  interpreter = Interpreter.load(model_directory)
  dict_ = interpreter.parse(text)
  dict_ = dict_['intent']['name']
  return dict_

def output(text):
  result = text.split('.')
  result = [r for r in result if len(r) > 0]
  output = []
  for i in result:
    output.append([i,intent_recog(i),entity_recog(i)])
  final = {"task_1_output" : text , "task_2_output" : [{"sentence": x, "intent":intent_recog(x),"entities":entity_recog(x)} for x in result]}
  return final


In [ ]:
text = "My name Lavkush and Ravi is I am calling from Amazon"
intent_recog(text)

'Intro'

In [ ]:
#Text Transcibed using VEED.io
text_1= """Hello.Hi, Nancy.This is Mike from AT&T incorporation. Yes.How can I help you? Nancy?You have been using our prepaid connection for a couple of years now, right?Yeah, that's right.How would you like a postpaid connection thatallows you to make free unlimited voice callsto three AT&T numbers?I would love that.But what's the catch?There's no catch.There will be a monthly rental which you willhave to pay like any other postpaid connection.Fantastic. Sign me up."""

In [ ]:
output(text_1)

{'task_1_output': "Hello.Hi, Nancy.This is Mike from AT&T incorporation. Yes.How can I help you? Nancy?You have been using our prepaid connection for a couple of years now, right?Yeah, that's right.How would you like a postpaid connection thatallows you to make free unlimited voice callsto three AT&T numbers?I would love that.But what's the catch?There's no catch.There will be a monthly rental which you willhave to pay like any other postpaid connection.Fantastic. Sign me up.",
 'task_2_output': [{'entities': [], 'intent': 'affirm', 'sentence': 'Hello'},
  {'entities': [{'entity_name': 'caller_name', 'entity_value': 'Nancy'}],
   'intent': 'affirm',
   'sentence': 'Hi, Nancy'},
  {'entities': [{'entity_name': 'caller_name', 'entity_value': 'Mike'},
    {'entity_name': 'company', 'entity_value': 'AT&T'}],
   'intent': 'Intro',
   'sentence': 'This is Mike from AT&T incorporation'},
  {'entities': [], 'intent': 'affirm', 'sentence': ' Yes'},
  {'entities': [],
   'intent': 'affirm',
   '

In [ ]:
import json
# convert into JSON:
y = json.dumps(output(text_1), indent=4)

# the result is a JSON string:
print(y)

{
    "task_1_output": "Hello.Hi, Nancy.This is Mike from AT&T incorporation. Yes.How can I help you? Nancy?You have been using our prepaid connection for a couple of years now, right?Yeah, that's right.How would you like a postpaid connection thatallows you to make free unlimited voice callsto three AT&T numbers?I would love that.But what's the catch?There's no catch.There will be a monthly rental which you willhave to pay like any other postpaid connection.Fantastic. Sign me up.",
    "task_2_output": [
        {
            "sentence": "Hello",
            "intent": "affirm",
            "entities": []
        },
        {
            "sentence": "Hi, Nancy",
            "intent": "affirm",
            "entities": [
                {
                    "entity_name": "caller_name",
                    "entity_value": "Nancy"
                }
            ]
        },
        {
            "sentence": "This is Mike from AT&T incorporation",
            "intent": "Intro",
           